## Part 1 - Scraping from Wiki 

In [ ]:
#Lets get set up

import requests 

import urllib.request

from bs4 import BeautifulSoup 

import pandas as pd 



In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [ ]:
data = requests.get(url).text

In [ ]:
soup = BeautifulSoup(data, 'html.parser')
tables =soup.find('table')
#print(tables)

In [5]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

#print(table_contents)

In [6]:
hoods = pd.DataFrame(table_contents)

In [7]:
hoods.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [8]:
hoods.shape

(103, 3)

## Part 2 - Getting Lat & Long 

In [ ]:
pip install geocoder

In [10]:
hoods.rename(columns={'PostalCode': 'Postal Code'}, inplace=True)

In [ ]:
hoods.head()

In [ ]:
# Geocoder was havng issues


import geocoder # import geocoder

lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
    for rows in hoods:
      g = geocoder.google('{hoods['Postal Code']}, Toronto, Ontario')
      lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    hoods.append(latitude, longitude)

In [ ]:
# Geocoder was havng issues

In [11]:
# The code was removed by Watson Studio for sharing.

In [13]:
df_data_1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
hoods_loc = pd.merge(hoods, df_data_1, on='Postal Code')

In [14]:
hoods_loc.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


## Part 3 - Explore & Cluster 

In [ ]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [ ]:
pip install folium

In [26]:
import folium

In [27]:
tor = hoods_loc[hoods_loc["Neighborhood"].str.contains("Toronto")]

In [28]:
tor.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
36,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
42,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576
73,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
80,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049
88,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321


In [39]:
!pip install geopy # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [40]:
# create map of New York using latitude and longitude values
map_can = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(tor['Latitude'], tor['Longitude'], tor['Borough'], tor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_can)  
    
map_can